In [321]:
import numpy as np
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [625]:
df=pd.read_csv("CT_Accidents.csv", low_memory=False)

In [673]:
for i in df.columns:
    print(df[i].isnull().values.sum())


df[features].isnull().sum()

0
0
0
0
0
0
0
0
0
0
0
25163
0
0
0
0
0
0
0
0
0
0
0
3640
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


Severity                   0
Temperature(F)             0
Humidity(%)                0
Pressure(in)               0
Visibility(mi)             0
Wind_Speed(mph)            0
Precipitation(tenth_in)    0
Weather_Condition          0
Amenity                    0
Bump                       0
Crossing                   0
Give_Way                   0
Junction                   0
No_Exit                    0
Railway                    0
Roundabout                 0
Station                    0
Stop                       0
Traffic_Calming            0
Traffic_Signal             0
Turning_Loop               0
Night                      0
dtype: int64

The original dataset included 49 states worth of accident data, so I restricted it to CT and saved it as a new csv so we don't have to load a 1Gb csv each time we open it. 

In [389]:
def comparison_plot(x,Y, Yhat):
    '''Plots Predicted vs True values for analysis of regression'''
    comparison_plot = figure(title='Difference between Measured and Predicted Values')
    comparison_plot.xaxis.axis_label='x'
    comparison_plot.yaxis.axis_label='Yhat-Y'
    comparison_plot.scatter(x=x,y=Yhat-Y)
    comparison_plot.line(x=[x.min(),x.max()],y=[0,0])
    return comparison_plot

This is a function that will be useful for our linear progression portion.

In [627]:
df.replace({False : int(0), True : int(1)}, inplace=True)

Lots of the features are boolean, so I just switched them to integers. 

In [628]:
df.replace({'Fair':int(0),'Clear':int(0), 
            'Partly Cloudy':int(1),'Scattered Clouds':int(1),'Haze':int(1),'Patches of Fog':int(1),
            'Cloudy':int(2), 'Mostly Cloudy':int(2), 'Overcast':int(2), 'Fair / Windy':int(2),'Mist':int(2),'Smoke':int(2),'Haze / Windy':int(2),
            'Light Rain':int(3),'Cloudy / Windy':int(3),'Fog':int(3), 'Light Drizzle':int(3), 'Thunder in the Vicinity':int(3),'Drizzle':int(3),'Light Freezing Fog':int(3),
            'Light Rain with Thunder':int(3),'Partly Cloudy / Windy':int(3),'Mostly Cloudy / Windy':int(3),'Fog / Windy':int(3),'Heavy Drizzle':int(3),'N/A Precipitation':int(3),
            'Rain':int(4),'Light Snow':int(4),'Heavy Rain':int(4),'Light Rain / Windy':int(4),'Snow':int(4),'Wintry Mix':int(4),'T-Storm':int(4),
            'Heavy T-Storm':int(4), 'Heavy Snow':int(4), 'Snow / Windy': int(4),'Heavy Rain / Windy':int(4),'Rain / Windy':int(4),'Thunder':int(4),
            'Light Freezing Rain':int(4),'Heavy T-Storm / Windy':int(4),'Light Sleet':int(4),'Ice Pellets':int(4),'Heavy Thunderstorms and Rain':int(4),
            'Light Thunderstorms and Rain':int(4), 'Thunderstorms and Rain':int(4),'Heavy Thunderstorms and Snow':int(4),'Light Ice Pellets':int(4),
            'Light Freezing Drizzle':int(4),'Heavy Snow / Windy':int(4),'T-Storm / Windy':int(4),'Thunderstorm':int(4),'Light Snow / Windy':int(4)}, inplace=True)
df['Weather_Condition'].value_counts()

0.0    13463
2.0     9138
3.0     2822
1.0     2691
4.0     1523
Name: Weather_Condition, dtype: int64

In [629]:
print(df['Wind_Chill(F)'].value_counts())
print(df['Wind_Chill(F)'].isnull().sum())



73.0    557
72.0    527
70.0    486
71.0    465
66.0    462
       ... 
20.6      1
30.2      1
11.6      1
21.5      1
12.2      1
Name: Wind_Chill(F), Length: 420, dtype: int64
4760


I've classified the weather condition features. These are kind of up to interpretation, and actually ended up making the score of the logistic regression slightly less accurate. These should probably be tweaked; then again, maybe there are other feature columns that really ought not be in our logistic regression that is throwing off our data. This dropped the score about 1-2%, and while its still in a decent range getting our accuracy closer to 75% would be best I think.

In [631]:
df['Sunrise_Sunset']=df['Sunrise_Sunset'].replace('Day',int(0))
df['Sunrise_Sunset']=df['Sunrise_Sunset'].replace('Night',int(1))
df.rename(columns={'Sunrise_Sunset':'Night'}, inplace=True)
df.iloc[:,44].value_counts()


0.0    18624
1.0    11125
Name: Night, dtype: int64

The 'Sunrise_Sunset' feature actually just tells us if something is night or day. These have been switched to integers.

In [633]:
df['Precipitation(in)']=df['Precipitation(in)']*10
df.rename(columns={'Precipitation(in)':'Precipitation(tenth_in)'}, inplace=True)


for i in df.columns:
    if df[i].dtype == float:
        df[i] = df[i].round().astype('Int64')

Here I switched the 'Precipitation' feature to be measured in tenths of an inch. This is because most of these values in inches are floats in a range of 0-2 inches, so when they are rounded and changed to integers the data is simplified to  values of either 0,1, or 2. By changing these values to tenths, we retain more accuracy in our data. This may be something we want to do with some other features, but I have not gone through all of them yet.

The loop is just changing every float value in each column to an integer.

In [634]:
df['Precipitation(tenth_in)'] = df['Precipitation(tenth_in)'].fillna(value=1000)
df.loc[(df['Weather_Condition']==int(4)) & (df['Precipitation(tenth_in)']==1000)  ,'Precipitation(tenth_in)'] = int(2)
df.loc[(df['Weather_Condition']!=int(4)) & (df['Precipitation(tenth_in)']==1000)  ,'Precipitation(tenth_in)'] = int(0)


In [468]:
rain = df.copy(deep=True)
print(rain['Precipitation(tenth_in)'].mean())
rain['Precipitation(tenth_in)']=df['Precipitation(tenth_in)'].replace(int(0),np.nan)

rain = rain.dropna(subset=['Precipitation(tenth_in)'])


rain['Precipitation(tenth_in)'].mean()



0.07941544885177453


1.9077231695085255

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29762 entries, 0 to 29761
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               29762 non-null  int64 
 1   ID                       29762 non-null  object
 2   Severity                 29762 non-null  int64 
 3   Start_Time               29762 non-null  object
 4   End_Time                 29762 non-null  object
 5   Start_Lat                29762 non-null  Int64 
 6   Start_Lng                29762 non-null  Int64 
 7   End_Lat                  29762 non-null  Int64 
 8   End_Lng                  29762 non-null  Int64 
 9   Distance(mi)             29762 non-null  Int64 
 10  Description              29762 non-null  object
 11  Number                   3630 non-null   Int64 
 12  Street                   29762 non-null  object
 13  Side                     29762 non-null  object
 14  City                     29762 non-nul

In [290]:
data=np.genfromtxt('CT_Accidents.csv',delimiter=',',skip_header=1)

In [657]:
features = df.columns[[2,22,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44]]
data = df[features].dropna()
data=data[features].to_numpy()



In [677]:
X = data[:,1:]
Y = data[:,0]
print(np.shape(X))
ones = np.ones((X.shape[0],1))
print(np.shape(ones))
x_new = np.hstack((X,ones))
print(np.shape(x_new))

(28552, 21)
(28552, 1)
(28552, 22)


Above I just quickly put together a data variable that takes our dataframe and formats it into arrays. I think this will be a necessary first step in preparing the data for Linear Regression.

In [637]:
df['Severity']=df['Severity'].replace(int(1),int(0))
df['Severity']=df['Severity'].replace(int(2),int(0))
df['Severity']=df['Severity'].replace(int(3),int(1))
df['Severity']=df['Severity'].replace(int(4),int(1))

df['Severity'].value_counts()


0    24766
1     4996
Name: Severity, dtype: int64

In [638]:
df['Severity'].value_counts()

0    24766
1     4996
Name: Severity, dtype: int64

Here I have reformated the 'Severity' feature that can act as our target. While we can use the original 4 classes for Linear Regression, restricting it to 2 will let us do our Binary Logistic Regression. I just did this after the Linear & PCA section, but we could always just make a new dataset out of this parameter.

In the original format, there are only 4 accidents with 'Severity'=1, so switching the classes to just two really shouldnt skew the data much at all. 

In [647]:
df = df.dropna(subset=['Temperature(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Weather_Condition','Night'])
print(df.isnull().values.any())


True


There's a good amount of nan values in each of our numerical feature columns. Here I just dropped every row with them, but we can assess which variables may be better to replace nan values with the average of that feature.

In [655]:
features = df.columns[[22,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44]]
L=LogisticRegression(max_iter=10000, solver='lbfgs')
x_train, x_test, y_train, y_test = train_test_split(df[features].values, df['Severity'].values)
L.fit(x_train,y_train)

LogisticRegression(max_iter=10000)

I just through in the numerical values into the logistic regression.

In [656]:
L.score(x_test,y_test)

0.8372093023255814

Around 70% is an okay start but hopefully we can improve. Another thing I haven't done yet is mess with the features that can be considered dependent variable. There might not be any though, since it looks like the variables dealing with time of day have been organized so there's probably no overlap. I havent formated those types of features to numerical values yet either. I feel like once we include those, our logistic regression should improve as well.

In [676]:
df[features].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28552 entries, 0 to 29761
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Severity                 28552 non-null  int64
 1   Temperature(F)           28552 non-null  Int64
 2   Humidity(%)              28552 non-null  Int64
 3   Pressure(in)             28552 non-null  Int64
 4   Visibility(mi)           28552 non-null  Int64
 5   Wind_Speed(mph)          28552 non-null  Int64
 6   Precipitation(tenth_in)  28552 non-null  Int64
 7   Weather_Condition        28552 non-null  Int64
 8   Amenity                  28552 non-null  int64
 9   Bump                     28552 non-null  int64
 10  Crossing                 28552 non-null  int64
 11  Give_Way                 28552 non-null  int64
 12  Junction                 28552 non-null  int64
 13  No_Exit                  28552 non-null  int64
 14  Railway                  28552 non-null  int64
 15  Ro